In [126]:
using DelimitedFiles 
using PyPlot
using Printf
import FlowFarm; const ff = FlowFarm

FlowFarm

# Amalia

## 36 directions

### Set Up

In [127]:
# include case information
include("./inputfiles/model_set_10_amalia_wind_park_36dir.jl");

### Run TIP Model

In [128]:
AEPTIP = ff.tip_get_aep(turbine_x,turbine_y,wind_data,cpdata,ctdata, pstep=0.01, prated=rated_power[1], u_cut_in=cut_in_speed[1], u_cut_out=cut_out_speed[1], density=air_density, r0=diam/2.0, alpha=0.01)




9.02390150255179e11

### Run FLOWFarm - Jensen TopHat Model

In [129]:
wakedeficitmodel = ff.JensenTopHat()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJTH = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

9.86846796586615e11

### Run FLOWFarm - Jensen Cosine Model

In [130]:
wakedeficitmodel = ff.JensenCosine()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJC = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

4.9790620008753064e11

### Run FLOWFarm - Bastankhah Model

In [131]:
wakedeficitmodel = ff.GaussYaw()
wakedeflectionmodel = ff.GaussYawDeflection()
wakecombinationmodel = ff.LinearLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelMaxTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPB = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

7.148770843360923e11

### Run FLOWFarm MultiZone Model

In [132]:
wakedeficitmodel = ff.Multizone()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPM = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

1.0419929594849794e12

### Compare TIP vs TopHat

In [133]:
ErrorTIPvsTopHat = 100*(AEPTIP-AEPJTH)/AEPJTH
ErrorTIPvsCosine = 100*(AEPTIP-AEPJC)/AEPJC
ErrorTIPvsMultizone = 100*(AEPTIP-AEPM)/AEPM
ErrorTIPvsBastankhah = 100*(AEPTIP-AEPB)/AEPB
println("TIP AEP: ", AEPTIP)
println("Jensen Top Hat AEP:   ", AEPJTH)
println("Jensen Cosine AEP:   ", AEPJC)
println("Multi-Zone AEP:   ", AEPM)
println("Bastankhah AEP:   ", AEPB)
@printf("Error vs Jensen Top Hat:  %.2f%%\n", ErrorTIPvsTopHat)
@printf("Error vs Jensen Cosine:   %.2f%%\n", ErrorTIPvsCosine)
@printf("Error vs Multi-Zone:  %.2f%%\n", ErrorTIPvsMultizone)
@printf("Error vs Bastankhah:   %.2f%%\n", ErrorTIPvsBastankhah)

TIP AEP: 9.02390150255179e11
Jensen Top Hat AEP:   9.86846796586615e11
Jensen Cosine AEP:   4.9790620008753064e11
Multi-Zone AEP:   1.0419929594849794e12
Bastankhah AEP:   7.148770843360923e11
Error vs Jensen Top Hat:  -8.56%
Error vs Jensen Cosine:   81.24%
Error vs Multi-Zone:  -13.40%
Error vs Bastankhah:   26.23%


## 72 directions

### Set Up

In [134]:
# include case information
include("./inputfiles/model_set_10_amalia_wind_park_72dir.jl");

### Run TIP Model

In [135]:
AEPTIP = ff.tip_get_aep(turbine_x,turbine_y,wind_data,cpdata,ctdata, pstep=0.01, prated=rated_power[1], u_cut_in=cut_in_speed[1], u_cut_out=cut_out_speed[1], density=air_density, r0=diam/2.0, alpha=0.01)

9.126619229740879e11

### Run FLOWFarm - Jensen TopHat Model

In [136]:
wakedeficitmodel = ff.JensenTopHat()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJTH = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

9.884343051400442e11

### Run FLOWFarm - Jensen Cosine Model

In [137]:
wakedeficitmodel = ff.JensenCosine()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJC = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

5.040595970872434e11

### Run FLOWFarm - Bastankhah Model

In [138]:
wakedeficitmodel = ff.GaussYaw()
wakedeflectionmodel = ff.GaussYawDeflection()
wakecombinationmodel = ff.LinearLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelMaxTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPB = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

7.295660197903009e11

### Run FLOWFarm MultiZone Model

In [139]:
wakedeficitmodel = ff.Multizone()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPM = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

1.0471662350436434e12

### Compare TIP vs TopHat

In [140]:
ErrorTIPvsTopHat = 100*(AEPTIP-AEPJTH)/AEPJTH
ErrorTIPvsCosine = 100*(AEPTIP-AEPJC)/AEPJC
ErrorTIPvsMultizone = 100*(AEPTIP-AEPM)/AEPM
ErrorTIPvsBastankhah = 100*(AEPTIP-AEPB)/AEPB
println("TIP AEP: ", AEPTIP)
println("Jensen Top Hat AEP:   ", AEPJTH)
println("Jensen Cosine AEP:   ", AEPJC)
println("Multi-Zone AEP:   ", AEPM)
println("Bastankhah AEP:   ", AEPB)
@printf("Error vs Jensen Top Hat:  %.3f%%\n", ErrorTIPvsTopHat)
@printf("Error vs Jensen Cosine:   %.3f%%\n", ErrorTIPvsCosine)
@printf("Error vs Multi-Zone:  %.3f%%\n", ErrorTIPvsMultizone)
@printf("Error vs Bastankhah:   %.3f%%\n", ErrorTIPvsBastankhah)

TIP AEP: 9.126619229740879e11
Jensen Top Hat AEP:   9.884343051400442e11
Jensen Cosine AEP:   5.040595970872434e11
Multi-Zone AEP:   1.0471662350436434e12
Bastankhah AEP:   7.295660197903009e11
Error vs Jensen Top Hat:  -7.666%
Error vs Jensen Cosine:   81.062%
Error vs Multi-Zone:  -12.845%
Error vs Bastankhah:   25.097%
